

1.   split_sequence 함수 정의: 시계열 데이터를 입력 시퀀스와 타겟 값으로 분할합니다.
2.   데이터 준비 및 전처리: 데이터를 정규화하고, 시퀀스를 생성한 후 훈련 및 테스트 세트로 분할합니다.
3. CNN+LSTM 모델 정의: 1D 컨볼루션 층, 맥스 풀링 층, LSTM 층, 그리고 덴스 층을 조합하여 모델을 구성합니다.
4. 모델 훈련 및 평가: 준비된 데이터로 모델을 훈련시키고 테스트 세트로 평가합니다.
5. 예측 및 역변환: 테스트 데이터에 대한 예측을 수행하고, 결과를 원래 스케일로 역변환합니다.
6. 결과 시각화: 학습 곡선과 예측 vs 실제 값을 그래프로 표시합니다.

주의사항


1.   데이터 파일 경로를 실제 데이터 파일의 경로로 수정해야 합니다.
2. n_steps와 n_features를 데이터의 특성에 맞게 조정해야 합니다.
3. 모델 아키텍처(레이어 수, 뉴런 수 등)를 데이터의 복잡성에 맞게 조정할 수 있습니다.
4. 에포크 수, 배치 크기 등의 하이퍼파라미터를 필요에 따라 조정할 수 있습니다.





In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. CSV 파일을 로드하고 전처리하는 함수
def load_and_preprocess_data(url):
    data = pd.read_csv(url)

    for column in data.select_dtypes(include=['object']):
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])

    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler

# 2. 시계열 데이터를 입력과 타겟으로 분할하는 함수
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# 3. CNN과 LSTM을 결합한 모델 생성 함수
def create_cnn_lstm_model(n_steps, n_features):
    model = Sequential([
        Input(shape=(n_steps, n_features)),
        Conv1D(filters=64, kernel_size=2, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=2, activation='relu', padding='same'),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 4. 모델을 훈련하고 평가하는 함수
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
    loss = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {loss}')
    return history

# 5. 메인 실행 함수
def main(url, n_steps=3):
    # 데이터 로드 및 전처리
    X, y, scaler = load_and_preprocess_data(url)

    # 시퀀스 데이터 생성
    X_seq, y_seq = split_sequence(X, n_steps)

    # 데이터 형태 출력 (디버깅용)
    print(f"X_seq shape: {X_seq.shape}")
    print(f"y_seq shape: {y_seq.shape}")

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

    # 모델 생성 및 훈련
    n_features = X_train.shape[2]
    model = create_cnn_lstm_model(n_steps, n_features)

    # 모델 구조 출력 (디버깅용)
    model.summary()

    history = train_and_evaluate_model(model, X_train, y_train, X_test, y_test)

    # 예측
    predictions = model.predict(X_test)

    print("훈련 완료. 테스트 데이터에 대한 예측값 일부:")
    print(predictions[:5])

if __name__ == "__main__":
    url = "https://raw.githubusercontent.com/MyungKyuYi/AI-class/main/abalone.csv"
    main(url)


X_seq shape: (4174, 3, 9)
y_seq shape: (4174, 9)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 3, 64)               │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 1, 32)               │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          16,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,995 (85.92 KB)

 Trainable params: 21,995 (85.92 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.8666 - val_loss: 0.6374
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6735 - val_loss: 0.6253
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7137 - val_loss: 0.6281
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6696 - val_loss: 0.6146
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6657 - val_loss: 0.6143
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6489 - val_loss: 0.6147
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6404 - val_loss: 0.6179
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6560 - val_loss: 0.6103
Epoch 9/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.6299 - val_loss: 0.6272
Epoch 10/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.6495 - val_loss: 0.6199
Epoch 11/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.6460 - val_loss: 0.6139
Epoch 12/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6